In [1]:
import os
import shutil
import pickle
import errno 

In [2]:
source_folder = 'databrut'

def delete_folder(folder_path):
    """Deletes a folder and its contents.

    Args:
        folder_path (str): The path to the folder to delete.
    """

    if not os.path.exists(folder_path):
        print(f"Folder '{folder_path}' does not exist.")
        return

    try:
        os.rmdir(folder_path)
        print(f"Folder '{folder_path}' deleted successfully.")
    except OSError as e:
        if e.errno == errno.ENOTEMPTY:
            print(f"Error deleting folder '{folder_path}': Folder is not empty.")
        else:
            print(f"Error deleting folder '{folder_path}': {e}")

# Dataset YOLO

In [3]:
fruit_list = sorted(os.listdir(source_folder))
fruit_to_idx = {fruit : i for i, fruit in enumerate(fruit_list)}

dataset = {
            'images': {
                'train': [],
                'val': []
            },
            'labels': {
                'train': [],
                'val': []
            }
        }

for fruit in fruit_list:
    images_path = os.path.join(source_folder, fruit)
    labels_path = os.path.join(images_path, 'Label')

    images = sorted([f for f in os.listdir(images_path) if f.endswith('.jpg')])[:400]
    labels = sorted([f for f in os.listdir(labels_path) if f.endswith('.txt')])[:400]

    if len(images) != len(labels):
        raise ValueError(f"Le nombre d'images et de labels ne correspond pas dans le dossier '{fruit}'")

    split_point = int(len(images) * 0.8)

    dataset['images']['train'].extend([os.path.join(images_path, img) for img in images[:split_point]])
    dataset['images']['val'].extend([os.path.join(images_path, img) for img in images[split_point:]])

    dataset['labels']['train'].extend([os.path.join(labels_path, lbl) for lbl in labels[:split_point]])
    dataset['labels']['val'].extend([os.path.join(labels_path, lbl) for lbl in labels[split_point:]])


In [4]:
# Chemin vers le dossier brut de données
data_brut_path = 'databrut'

# Chemin vers le dossier où vous voulez créer la structure finale
dataset_path = os.path.join('datasets', 'yolo')

# supression des dossiers images et labels avant de les remplir
delete_folder(os.path.join(dataset_path, 'images'))
delete_folder(os.path.join(dataset_path, 'labels'))

# Créer les répertoires nécessaires
def create_directories(base_path):
    os.makedirs(os.path.join(base_path, 'images/train'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'images/val'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'labels/train'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'labels/val'), exist_ok=True)

# Copier les fichiers d'images et de labels
def copy_files(src_paths, dest_base_path, subfolder):
    for src in src_paths:
        src_file = src
        relative_path = os.path.relpath(src_file, start=data_brut_path)
        dest_file = os.path.join(dest_base_path, subfolder)
        dest_folder = os.path.dirname(dest_file)
        os.makedirs(dest_folder, exist_ok=True)
        shutil.copy(src_file, dest_file)

# Créer la structure de répertoires
create_directories(dataset_path)

# Copier les fichiers d'images et de labels
for split in ['train', 'val']:
    copy_files(dataset['images'][split], dataset_path, f'images/{split}')
    copy_files(dataset['labels'][split], dataset_path, f'labels/{split}')

Error deleting folder 'datasets/yolo/images': Folder is not empty.
Error deleting folder 'datasets/yolo/labels': Folder is not empty.


In [ ]:
def modify_txt_files(directory, fruit_to_id):
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r") as f:
                lines = f.readlines()  
  

            new_lines = []
            for line in lines:
                values = line.strip().split()
                if len(values) == 6:
                    # Cas particulier pour "Bell pepper"
                    class_name = "Bell pepper"
                    _, _,x, y, w, h = values
                else:
                    class_name, x, y, w, h = values
                
                class_id = fruit_to_id.get(class_name, -1)  # Get ID, default -1 if not found
                if class_id != -1:  # Only modify if class is found
                    new_line = f"{class_id} {x} {y} {w} {h}\n"
                    new_lines.append(new_line)

            with open(filepath, "w") as f:
                f.writelines(new_lines)

# Replace "your_directory" with the actual path to your directory
modify_txt_files("datasets/yolo/labels/train", fruit_to_idx)
modify_txt_files("datasets/yolo/labels/val", fruit_to_idx)
